In [130]:
import json
import pandas as pd

In [131]:
intent_df = pd.read_csv('../data/intent.csv')
entities_df = pd.read_csv('../data/entities.csv')
entities_df = entities_df[entities_df.word.apply(lambda x: len(x)) >= 2]
entities_df = entities_df[entities_df.word.apply(lambda x: len(x)) <= 4]
entities_df.to_csv('../data/entities.csv', index=False)

In [132]:
new_words = sorted(entities_df.word.tolist(), key=lambda x: -len(x))
with open('../jieba_userdict/jieba_userdict.txt', 'w', encoding='utf8') as f_handle:
    f_handle.writelines([w + '\n' for w in new_words])

In [133]:
entities_df['len'] = entities_df['word'].apply(lambda x: len(x))
entities_df = entities_df.sort_values('len', ascending=False)

In [134]:
question = "自动更新" #intent['questions'][125]
intent = '测试'

In [135]:
def extract_entities(question, intent, entities_df):
    data = {}
    find_entities = []
    already_in = []
    for i, e in entities_df.iterrows():
        word = e['word']
        s = question.find(word)
        if s >= 0:
            for w in already_in:
                if w.find(word) >= 0:
                    break
            else:
                already_in.append(word)
                find_entities.append({'value': word, 'entity': e['entity'], 'start': s, 'end': s + len(word)})

    data['text'] = question
    data['intent'] = intent
    data['entities'] = find_entities
    return data

In [136]:
common_data = []
for i, row in intent_df.iterrows():
    question = row['questions']
    intent = row['intention']
    common_data.append(extract_entities(question, intent, entities_df))

In [137]:
nlu_data = {}
nlu_data['rasa_nlu_data'] = {}
nlu_data['rasa_nlu_data']['common_examples'] = common_data
nlu_data['rasa_nlu_data']['regex_features'] = []
nlu_data['rasa_nlu_data']['lookup_tables'] = []
nlu_data['rasa_nlu_data']['entity_synonyms'] = []

In [138]:
json.dump(nlu_data, open('../data/nlu_data.json', 'w'))